# Predictors Modeling and Energy

This notebook provides the code to:

1. Model with Alphafold2-Multimer the desired target given a fasta file

2. Relax the models created Alphafold2-Multimer

3. Extract, convert and create a pseudo-log.txt from AF3 models

4. Calculate the bind Energy of Alphafold2-Multimer and AF3 models

There are two alternative codes A and B. A is for Local calculations and B for calculations using MN5 (Cluster)

# Settings of the target

## Libraries

In [ ]:
# File manegement
import os, zipfile 
import re 
import shutil

# Data manegement
import pandas as pd # used to manage dataframes
import numpy as np
from itertools import product
from Bio import PDB
from Bio.PDB import MMCIFParser, PDBIO, DSSP, NeighborSearch,Superimposer,PDBParser
from Bio.Align import PairwiseAligner
from scipy.spatial.transform import Rotation as R
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
# Subprocess to calling bash
import subprocess # used to call bash and running external programs like pydock4

## Target election and p_type

In this section, you choose:

- The target.

- The template to copy the existing scripts.

- The folder where we collect the fastas for modeling.

- The test settings (Predictors or Scorers) and whether it is local or on MareNostrum.


In [ ]:
# Definicion del directorio principal
dir_capri=""

#Plantilla  
nplantilla="T901" 

# Seleccion de target
target="T902" 

### Settings
p_type= "Predictors"

# Definicion de directorios
fasta_dir=os.path.join(dir_capri,"fastas") # Where the fastas are gatherer
dir_target=os.path.join(dir_capri,target) # Directory of the complete target
dir_template=os.path.join(dir_capri,nplantilla)# Template with all the necesary scripts
dir_complex=f"{dir_target}/Predictors/AF_MODELS/COMPLEX" # This directory is needed to execute several codes

# Definicion de archivos fasta
original_fasta=f"{fasta_dir}/{target}.fasta"
fasta=f"{dir_complex}/{target}.fasta"



## Creation of the target folder 

En el siguiente bloque copiamos la plantilla, con un nuevo nombre. Después el fasta de la carpeta donde se almacenan (fasta_dir)

In [ ]:
# First, ensure that the fasta file is in fasta_dir

# Create the target folder with all the scripts from the template if the fasta exists
if not os.path.exists(dir_target):
    shutil.copytree(dir_template, dir_target)
    print(f"Folder created: {dir_target}")
else:
    print("The folder already exists")

if os.path.exists(original_fasta):
    shutil.copy2(original_fasta, fasta)
    print(f"File {original_fasta} copied to {fasta}") 
else: 
    print(f"The fasta doesn't exist in {original_fasta} or {dir_target} doesn't exist")
    assert False, f"Place the fasta file in {fasta_dir}"

# Ensure that the file is inside the target folder
# Remember that the original_fasta needs to be in fasta_dir to copy correctly

if os.path.exists(fasta):
    print("Existing file", fasta)
else: 
    print("File does not exist", fasta)
    assert False, f"Warning, the file was not copied correctly, fasta is not in {fasta}"



In [ ]:
assert False, "Template copied, now go to local or to server"

# A Local Method

## 1 Alphafold2-Multimer modeling (LOCAL)

In [ ]:
if local:
    print("local")
    subprocess.run(['bash',"conda activate Alphafold2\n","bash ./script_calculo_local.sh"])

## 2 Relaxation (LOCAL)

In [ ]:
import os
import subprocess
import glob

def relaxation(dir_complex, target):


    greasy = 'greasy'
    colabfold_relax = 'colabfold_relax'
    
    # Creation of each command of relaxation to pass to shell script
    os.chdir(dir_complex)
    print(os.getcwd())
    os.chdir('..')
    complex_dir = os.getcwd()
    fasta_files = glob.glob(os.path.join(complex_dir, '*.fasta'))
    for fasta_file in fasta_files:
        base_name = os.path.basename(fasta_file).replace('.fasta', '')
        cmd_file = os.path.join(complex_dir, f'{base_name}.colabfold.relax.cmd')
        
        with open(cmd_file, 'w') as cmd_fp:
            cmd_fp.write(f'#!/bin/bash\n{greasy} {base_name}.colabfold.relax.greasy\n')
        
        pdb_files = glob.glob(os.path.join(complex_dir, 'T*_*.r*.pdb'))
        
        with open(cmd_file, 'a') as cmd_fp:
            for pdb_file in pdb_files:
                cmd_fp.write(f"{colabfold_relax} --max-iterations 2000 --tolerance 2.39 --stiffness 10.0 --max-outer-iterations 3 --use-gpu {pdb_file} {pdb_file.replace('/COMPLEX', '')}\n")

    # Relaxation execution
    os.environ['GREASY_NWORKERS'] = '2' # change if you need more, keep in mind that more workers could lead to abortion of execution
    os.chdir(complex_dir)
    file_path=f'{os.getcwd()}/COMPLEX/{target}.colabfold.relax.cmd'# Check if the file is correcly created
    return file_path
   
file_path=relaxation(dir_complex, target)


Check that CMD is correctly generated 

In [ ]:
cmd= open(file_path)
content= cmd.read()
print(cmd)

In [ ]:
assert False, "Look at the script, comment this line if you don't want to pause the execution"

In [ ]:
# Execution of the relaxation
subprocess.run(['bash', file_path])

## 3. AF3 models processing (LOCAL)

### Preparation of Alphafold3 Models
1. Decompression

2. Conversion to PDB

3. Extraction of data from the JSON: iptm and ptm to save the model confidence in an artificial log.txt, similar to the one from Alphafold Multimer

### 3.1 Decompression

<div style="border-left: 4px solid; background-color:#FF6347; color: #FFFFFF; padding: 10px;">
    <strong style="color: #FF6347;">Nota:</strong>
    <span style="color: #000000;">WARNING: rebember to download the AF3 models and have them in the dir_complex</span>
</div>



In [ ]:
# Uncompress the AlphaFold3 Job.
def descomprimir_archivo(zip_path, directorio_destino):

    # Ensure that the destination directory exists; if not, create it
    if not os.path.exists(directorio_destino):
        os.makedirs(directorio_destino)

    # Open the ZIP file in read mode
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all files into the specified directory
        zip_ref.extractall(directorio_destino)

patron = r'(.*(\d+)\.zip$)'
patron_CIF = r'(.*(\d+)\.cif$)'

zipfiles = [os.path.abspath(os.path.join(dir_complex, archivo)) for archivo in os.listdir(dir_complex) if re.match(patron, archivo)]
print(zipfiles)
for zipfille in zipfiles:
    print(zipfille)
    descomprimir_archivo(zipfille, zipfille.rstrip('.zip'))

CIF_files = [os.path.abspath(os.path.join(dir_complex, archivo)) for archivo in os.listdir(zipfille.rstrip('.zip')) if re.match(patron_CIF, archivo)]


### 3.2 Conversion to PDB

In [ ]:
#Convert CIF files to PDB files of the AplhaFold3 Job.
def convert_cif_to_pdb(cif_file, pdb_file):
    """
    Convert a CIF file to a PDB file using Biopython.

    Parameters:
    cif_file (str): Path to the input CIF file.
    pdb_file (str): Path to the output PDB file.
    """
    parser = MMCIFParser()
    structure = parser.get_structure('ID', cif_file)
    io = PDBIO()
    io.set_structure(structure)
    io.save(pdb_file)

patron_CIF = r'(.*(\d+)\.cif$)'
for zipfille in zipfiles:
    CIF_files = [os.path.join(zipfille.rstrip('.zip'),archivo) for archivo in os.listdir(zipfille.rstrip('.zip')) if re.match(patron_CIF, archivo)]
    #print(CIF_files)
    for CIF_file in CIF_files:
        #print(CIF_file)
        convert_cif_to_pdb(CIF_file, CIF_file.replace('.cif','.pdb'))

### 3.3 JSON data extraction

In [ ]:
#Read the Json
import os
import json

def leer_json_extract_vars(directorio, claves):
    """
    Lee archivos JSON en un directorio específico y extrae las variables especificadas.
    
    Parámetros:
    directorio (str): Ruta al directorio que contiene los archivos JSON.
    claves (list): Lista de claves a extraer de los archivos JSON.
    
    Retorna:
    dict: Diccionario con nombres de archivo y sus variables extraídas.
    """
    resultados = {}  # Diccionario para almacenar los resultados

    # Recorrer todos los archivos en el directorio
    pattern_json = re.compile(r"summary_confidences_\w\.json$")
    for archivo in os.listdir(directorio):
        if pattern_json.search(archivo):  # Asegurarse de que es un archivo JSON
            ruta_completa = os.path.join(directorio, archivo)
            with open(ruta_completa, 'r') as f:
                data = json.load(f)  # Cargar el contenido JSON
                # Extraer las variables especificadas
                valores_extraidos = {clave: data.get(clave, None) for clave in claves}
                
                # Almacenar los resultados
                resultados[archivo] = valores_extraidos

    return resultados

# Usar la función
claves_a_extraer = ['ptm', 'iptm']  # Añadir aquí cualquier clave que necesites
for zipfille in zipfiles:
    log_folder = zipfille.rstrip('.zip')
    resultados = leer_json_extract_vars(zipfille.rstrip('.zip'), claves_a_extraer)
    with open(os.path.join(log_folder,'log.txt'), 'w') as file:
        for archivo, vars in resultados.items():
            # Formatear nombre del archivo y eliminar partes no deseadas
            nombre_archivo_formateado = archivo.replace('summary_confidences', 'model').rstrip('.json')
            # Crear una cadena de texto con los pares clave=valor
            vars_text = ' '.join([f"{key.replace('tm','TM')}={value}" for key, value in vars.items()])
            file.write(f"{nombre_archivo_formateado} {vars_text}\n")

In [ ]:
#subprocess.run(['bash',f"scp -r fold_t282_1 bsc054620@glogin1.bsc.es:/gpfs/projects/bsc54/Capri/{target}/Predictors/AF_MODELS/COMPLEX/"])


## 4. pyDock Bind Energy (Local)

### 4.1 Ligand and receptor asignation

In [ ]:
multip=True
if multip:
    script_content="""
 #!/bin/bash
        #rm Greasy_BindE_mul_ligs.txt;
        CHAINS_LIG_VALUES=(
            "B,C,D A"
            "A,C,D B"
            "A,B,D C"
            "A,B,C D"
            "A E,G"
            "B I,K"
            "C F,H"
            "D J,L"
        )

        for h in $(ls -d  T*cola*/ fold*/);do
                echo $h;
                cd $h;
                for j in *.pdb;do
                        echo $j
                        for value in "${CHAINS_LIG_VALUES[@]}"; do
                    CHAINS_LIG="$value"
                    LIG=echo $CHAINS_LIG | cut -d" " -f 2
                            REC=echo $CHAINS_LIG | cut -d" " -f 1
                            CH=${LIG/,}
                            echo -e "[receptor]\npdb         = $j\nmol         = $REC\nnewmol      = $REC" > ${j/.pdb}_LIG_${CH}.ini;
                            echo -e "[ligand]\npdb         = $j\nmol         = $LIG\nnewmol      = $LIG" >>  ${j/.pdb}_LIG_${CH}.ini;
                            #echo -e "[reference]\npdb         = ranked_0_REF.pdb\nrecmol      = $REC\nligmol      = $LIG\nnewrecmol   = $REC\nnewligmol   = $LIG\n" >> ${j/.pdb}_LIG_${CH}.ini;
                            echo "cd ${h}; timeout 5m pydock4 ${j/.pdb}_LIG_${CH} bindEy;cd -" >> ../Greasy_BindE_mul_ligs.txt;
                done
                done
                cd -;
        done
        """

    comando=f"cd {dir_complex}\n bash {script_content}"
    #subprocess.run(['bash',f"{comando}"])
else:
    comando=f"cd {dir_complex}\n bash ./ini_creator_bindE.sh"
    subprocess.run(['bash',f"{comando}"])

### 4.2 Bind Energy calculation

In [ ]:
comando=f"cd {dir_complex}\n bash ./script_calculo_local.sh"
if local:
    subprocess.run(['bash', f"{comando}"])

### 4.3 Energy summation

In [ ]:
comando=f"cd {dir_complex}\n bash ./sum_ene_multy_bindEy_new.sh"
if local:
    subprocess.run(['bash',f"{comando}"])